# 1. Abstract

Unbalanced data remains a major problem in the field of data-mining. It is very popular among medical datasets where the number of positive cases of patients who the conditions is much smaller than those who don't. This imbalance causes many learning method to bias the negative cases and predict the majority of cases to be negative. These learning methods have a very high "accuracy" due to an abundance of negative cases, but are extremely poor at predicting positive cases.

Common methods to fix this problem include removing negative samples that are irrelevant, ignore irrelevent attributes or artificially create new instances of positive cases (known as oversampling). Oversampling methods (such as SMOTE) generally perform better than others **[citation]**, but the increase in performance varies across datasets. In this paper, we aim to examine various oversampling methods, include the one we proposed. We focus on visually described how one method might increase the prediction performance. We then test all methods, including our own, using various medtrics. Later on, we will describe a novel tool that we build to specifically examine the behavior of various oversampling methods in unbalanced datasets.

# 2. Introduction

Imbalanced dataset has been shown to have negative impact on the many classification methods. The most notable effect of imbalance dataset is that it skew the classification mechanism into biasing the majority class. Because there are an overwhelming number of instances of the majority class, the algorithm tends to predict more majority classes so that it can achieve higher "accuracy". This behavior is undesirable. For examples, if we have a dataset of 2 malignant cancer tumor cases and 98 benign cases, a method predicting all cases to be negatives will have 98% "accuracy", but has no capability to different positive cases from negative cases. Thus, evaluating "accuracy" is  generally irrelevant when there is significant imbalance between classes in the dataset.

!["Confusion Matrix"](./Images/confusion_matrix.png "ROC curve for both methods")

A common method to resolve this issue is to separate the performance of positive cases and negative cases and build a "confusion matrix." Using confirmation matrix, we can easily see if the algorithm can actually perform well on each separate class. After making the matrix, we can then calculate various other medtrics. Two common medtrics derived from the confusion matrix is:

* **True Positive Rate**: (TP / TP + FN) This medtrics assess how well the methods can classify among actual positve class.
* **False Positive Rate**: (FP / FP + TN) This medtrics assess how terrible the methods does among classifying actual negative class.

Generally speaking, we would want a method to have high True Positive Rate and at the same time low False Positive Rate. A method behaves as such means it can classify a lot of positive instances right while not predicting too many negative instances wrong.

Even the True Positive Rate and False Positive Rate can be misleading. Most learning methods produce a probability of each class, and the prediction can be modified by a "threshold value." The figure below demonstrates a hypothetical dataset and its probability prediction among each class. As we can observe, "true positve rate" and "false positive rate" may vary depending on the threshold values. Researchers can "cherry-pick" a threshold that seems to produce results that look good but have little values when comparing one method against each other.

| Class    |    0 |    0 |   0 |    0 |    0 |    0 |    0 |    0 |    0 |    0 |    1 |    1 |    1 |    1 |    1 |    1 |    1 |    1 |    1 |    1 |
|----------|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
| Method 1 | 0.25 |  0.4 | 0.1 | 0.05 |  0.7 | 0.65 | 0.22 | 0.48 |  0.7 | 0.27 |  0.9 | 0.75 |  0.8 | 0.55 | 0.65 | 0.44 | 0.37 | 0.56 | 0.27 | 0.98 |
| Method 2 |  0.4 | 0.15 | 0.3 | 0.03 | 0.34 | 0.78 | 0.42 | 0.23 | 0.88 | 0.22 | 0.74 | 0.78 | 0.82 | 0.25 | 0.55 | 0.73 | 0.83 | 0.42 | 0.99 |  0.7 |
<center>*Table 1: A hypothical dataset with prediction from 2 hypothetical methods*</center>

| Threshold | 0 |  0.1 | 0.2 | 0.3 |  0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1 |
|-----------|--:|-----:|----:|----:|-----:|----:|----:|----:|----:|----:|--:|
| TPR       | 1 |    1 |   1 | 0.9 |  0.8 | 0.7 | 0.5 | 0.4 | 0.3 | 0.2 | 0 |
| FPR       | 1 | 0.89 | 0.8 | 0.5 | 0.45 | 0.4 | 0.4 |   0 |   0 |   0 | 0 |
<center>*Table 2: Different TPR and FPR values for Method 1*</center>


| Threshold | 0 | 0.1 | 0.2 | 0.3 |  0.4 | 0.5 | 0.6 |  0.7 | 0.8 | 0.9 | 1 |
|-----------|--:|----:|----:|----:|-----:|----:|----:|-----:|----:|----:|--:|
| TPR       | 1 |   1 |   1 | 0.9 |  0.9 | 0.8 | 0.7 | 0.67 | 0.3 | 0.1 | 0 |
| FPR       | 1 | 0.9 | 0.8 | 0.5 | 0.33 | 0.2 | 0.2 |  0.2 | 0.1 |   0 | 0 |
<center>*Table 3: Different TPR and FPR values for Method 2*</center>

For example, if we choose threshold 0.8 for both method. Then it seems that Method 1 performs better than Method 2 since they have the same TPR (0.3) but lower FPR (0 compared to 0.1). If we, however, choose threshold 0.5, then Method clearly performs better than method 1 since it has higher TPR and lower FPR. If we choose threshold 0.3, then two methods would perform equally well. Therefore, TPR and FPR alone is not enough to compare the performance of two methods.

To prevent this, another method called "Area Under Curve" is implemented. Based on all probability predictions, the method will calculate all possible pair of (TPR, FPR) values and map them as a curve. We then proceed to calculate the area under the curve. A good learning method should maintain a high TPR even when the threshold starts to favor negative cases and, thus, should have higher AUC values in general. As we can see below, it is now clear that Method 2 actually performs better than Method 1 since it has a bigger AUC. We will mainly use AUC as the main value to judge the performances of difference over sampling methods accross different datasets.

!["ROC curve for both methods"](./Images/ROC_curves.png "ROC curve for both methods")

# 3. Visualize behavior of different oversampling methods

!["OriginalData"](./Images/TrueOriginalData.png "Logo Title Text 1")

In this section, we will try to visualize the behavior of each sampling method through a hypothetical dataset. This dataset consists of 10000 negative cases and 300 positive cases in three separate circles with theoretical boundary denoted as the dashed line. One might ask why we don't create each case for each circle. The reasoning is simple: most datasets are hyper-dimensional (more than 3 attributes) and since there is no way to visualize more than 3 dimensions, it becomes extremely difficult for pracititioners to separate these classes by hands, even though different cases might be sporadic. Therefore, we expect the oversampling methods to also adapt to this situation where positive cases scatter into different groups.

[Maybe provide a few more hypothetical datasets]

## SMOTE

SMOTE is a very common over-sampling method when dealing with unbalanced dataset. You can read the pseudo-code in the original paper by **[citation]**. Essentially, the method pick a random point across the positive cases. It then picks k nearest neighbors of the same classes with the point in question. It then proceeds to choose two random points among these nearest neighbors and simply create a new positive instance somewhere in the middle.

!["OriginalData"](./Images/ComparisonSMOTE.png "Logo Title Text 1")

As we can see, SMOTE seems to "strengthen" the border by creating many instances between neighboring points. This behavior can be very beneficial for distance-related methods such as kNN. As you can see, the articial positive instances help the articial boundary approach closer to actual theorical boundary.

When k is too small, this "bordering behavior" becomes limited because it does stretch to points that are far across. When k is too big, however, a point might incidentally pick a neighbor that is actually among a separate group, potentially create some awkward links. In the example above, the local group on the bottom right only has 9 instances, but any random point has to pick 10 nearest neighbors. Point A, therefore, might pick a point that is extremely far apart, but still consider one of its nearest neighbors. This far apart point can in turn be chosen to create new a new instance, potentially create an awkard link in the middle.

The most desirable option is to choose the biggest k that is still smaller than the number of instances of the smallest local groups. In practice, this is hard to achieve, since there is no way to visualize more than 3 dimensions to actually know which groups are "local." Optimal k value is achieved on a case by case basis through experimentation.

[that is k, what about the number of samples generated?]

## Borderline SMOTE (Coming soon)

## AGO

Because of this awkward linkage weakness in SMOTE, we devise a new method to mitigate this problem called Augmented Gaussian Sampling (AGO). We provide the pseudo-code of the method in the reference section. Basically, we pick a random point and then choose k nearest neighbors. We then generate a new point using a Normal distribution N(mu, sigma) with mu equals to the point in question and sigma is the standard deviation of k nearest data points. This way, we essentially create a new point around the original positive case we initially pick.

!["OriginalData"](./Images/ComparisonAGO.png "Logo Title Text 1")

When a lot of these points are generated, the method has the effect of strengthening local points by creating points surrounding these locals.

This weakness might still suffer from the value of k that is too big. Similar to SMOTE, this method can sometimes choose an instance that is too far away from the original point and this instance can blow up the standard deviation and create a new instance that is slightly far away from the original point. However, since the new points are still close to the original points, its spill over effect is significantly less harmful than SMOTE.

Another weakness of this method is that a lot of times, the random point is already way inside the decision boundary and generating new instances around these points doesn't have much effect on the overall result. This effected can be solved by using Borderline NGO.

## Borderline NGO (Unfinished)

The obvious way to avoid oversampling these inside points is to only oversample points that are closed to the borders. To determine which instance is closed to tihe border, we can select k nearest points (instead of k nearest positive instances). If all k nearest points are positive, then we can be fairly certain that this point is already way inside the positive territory and there is no need to oversample this point. If there is at least c points (1 <= c <= k) that are negative, we can suspect that the point is very close to the border and needs to be oversampled.

We can see visually this method will clearly help reduce the abundant oversampled instances and only strengthens "borderline" points. The problem with this method is that computational cost of generating new instances will blow up. This is because since the cost of k-nearest neighbors algorithm is O(n^2), it is much more expensive to apply kNN on the entire dataset than to just on the positive instances (which are assumed to be minimal compared to the negative instances. We will examine the performance results to see if the increase in performance (if at all) is worth the blown up cost.

# 4. Testing performance among different oversampling methods.

## Method

To test the performance of different oversampling methods, we use 7 different learning algorithms to see how much one oversampling method might or might not enhance the performance of a learning algorithm. We will test each learning algorithm first on the original dataset, and then for each new dataset that contained oversampling positive instances from each of the described oversampling methods. The AUC value is generated through 10-fold cross-validating the dataset 100 times to get an average value and a confidence intervals.

The following datasets are used for performance testing:

* **kDD 2008 Breast Cancer Classification:** Official dataset of kDD Cup 2008. The dataset contains 623 positive instances and 101671 negative instances and is, thus, heavily imbalanced. 117 attributes were generated from the original MRI through an image processing algorithm, but no additional information about these features were released for confidentiality reason.
* **Phoneme Dataset:** The dataset contains 3818 instances of nasal sounds and 1586 instances of oral sounds. The dataset has 5 attributes.
* **Satellite Image Dataset:** There are 6 classes in this dataset. We choose class 4 as the positive class and combine all instances of other classes to form negative class. This way, there are 5809 instances of positive classes and 626 instances of negative classes.

|                                | Number of Attributes | Positive Instance | Negative Instance | Positive / Negative Ratio |
|--------------------------------|----------------------|-------------------|-------------------|---------------------------|
| kDD 2008 Breast Cancer dataset | 117                  | 623               | 101671            | 1 : 163.2                 |
| Phoneme Dataset                | 5                    | 1586              | 3818              | 1 : 2.43                  |
| Satellite Image Dataset        | 36                   | 626               | 5809              | 1 : 9.28                  |
<center>*Table 1: Information about dataset used*</center>

## Results

| Method                          | No oversampling | Our method (kn = 30) | SMOTE (kn = 30) |
|---------------------------------|-----------------|----------------------|-----------------|
| Logistic Regression             | 0.930           | **0.945**                | 0.936           |
| Decision Tree                   | 0.782           | **0.837**                | 0.741           |
| Random Forest (k = 100)         | 0.909           | 0.917                | **0.945**           |
| SVM RBF                         | 0.843           | **0.935**                | 0.933           |
| Gaussian Naive Bayes Classifier | **0.875**           | 0.863                | 0.836           |
| AdaBoost - Base Tree            | 0.895           | 0.900                | **0.910**           |
| Deep Neural Network             | Coming soon     | Coming soon          | Coming soon     |
<center>*Table 2: AUC value for Breast Cancer Classification problem*</center>

| Method                          | No oversampling | Our method (kn = 3) | SMOTE (kn = 3) |
|---------------------------------|-----------------|---------------------|----------------|
| Logistic Regression             | **0.821**           | 0.820               | 0.820          |
| Decision Tree                   | 0.881           | **0.877**               | **0.877**          |
| Random Forest (k = 100)         | 0.955           | **0.957**               | 0.954          |
| SVM RBF                         | 0.900           | 0.907               | **0.908**          |
| Gaussian Naive Bayes Classifier | 0.820           | **0.821**               | 0.819          |
| AdaBoost - Base Tree            | **0.925**           | 0.910               | 0.917          |
| Deep Neural Network             | Coming soon     | Coming soon         | Coming soon    |
<center>*Table 3: AUC Value for Phoneme Classification problem*</center>

| Method                          | No oversampling | Our method (kn = 30) | SMOTE (kn = 30) |
|---------------------------------|-----------------|----------------------|-----------------|
| Logistic Regression             | **0.768**           | **0.768**                | 0.767           |
| Decision Tree                   | 0.826           | **0.861**              | 0.835           |
| Random Forest (k = 100)         | 0.960           | **0.966**                | 0.964           |
| SVM RBF                         | 0.900           | 0.888                | 0.911           |
| Gaussian Naive Bayes Classifier | **0.915**           | **0.915**                | 0.912           |
| AdaBoost - Base Tree            | 0.944           | **0.951**                | 0.944           |
| Deep Neural Network             | Coming soon       | Coming soon           | Coming soon       |
<center>*Table 4: AUC Value for Satellite Image Classification problem*</center>


The result clearly shows that oversampling methods generally increase the performance of the learning algorithm. AGO and SMOTE have a relative similar increases in performance. Whichever method performs better seems to be on a case by case basis.

Random Forest algorithm generally performs well even with the original dataset, suggesting that the learning mechanism is less affected by the imbalance of the dataset. There is still, however, some performance improvement when the dataset is pre-processed by some oversampling methods.